In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from past.builtins import xrange
import datetime
from datetime import timedelta
from pandas import DataFrame
from scipy.fftpack import fft, ifft,dct
import pywt
import scipy.stats as stats

In [21]:
#------Please Enter name of TestFile here as "nameoffile.csv"--------
data = pd.read_csv('test.csv', error_bad_lines=False)
data = data.values.tolist()

FileNotFoundError: ignored

In [22]:
 def rank_feature(meal_cgm_master_list):
  def calc_rank_row(meal_data,row):
    rank = [0 for x in range(len(meal_data[row]))]
    for j in range (len(meal_data[row])):
      (r,s)=(1,1)
      for k in range (len(meal_data[row])):
        if k != j and meal_data[row][k] < meal_data[row][j]:
          r += 1
        if k != j and meal_data[row][k] == meal_data[row][j]:
          s += 1       
      rank[j] = r + (s - 1) / 2
    return (rank) 

  rank_list_meal=[]
  for row in range(len(data)):
    ranks_list_meal = calc_rank_row(data,row)
    rank_list_meal.append(ranks_list_meal)
  #print(rank_list_meal)
  ranks_list_df_meal = pd.DataFrame(rank_list_meal)
  ranks_list_df_meal = ranks_list_df_meal.iloc[:,:24]
  return (ranks_list_df_meal)

In [23]:
def z_score_feature(meal_cgm_master_list):
  def z_score_feat(meal_list,row):
    z_score_arr = np.array(meal_list[row])
    z_score_list = stats.zscore(z_score_arr)
    return (z_score_list)

  z_scores_meal=[]
  for row in range (len(data)):
    z_score_res_list = z_score_feat(data,row)
    z_scores_meal.append(z_score_res_list)
  z_score_res_list_df_meal = pd.DataFrame(z_scores_meal)
  z_score_res_list_df_meal = z_score_res_list_df_meal.iloc[:,:24]
  return (z_score_res_list_df_meal)

In [24]:
def peaks_fouriertrans_feature(meal_cgm_master_list):
  def calc_peaks_fouriertrans(meal_list,row):
      fouriertrans = list()
      peak_feat = list()
      arr = np.array(meal_list[row])
      # print(fft(arr))
      fouriertrans.append(abs(fft(arr)))
      for val in range(len(fouriertrans)):
        sets = set(fouriertrans[val])  
        set_list = list(sets)
        set_list.sort()
        set_list = set_list[::-1][0:8]
        # print(set_list)
        peak_feat+=set_list
      # print(peak_feat)
      return (fouriertrans,peak_feat)  


  x_meal=[]
  y_meal=[]
  for row in range(len(data)):
    x1,y1 = calc_peaks_fouriertrans(data,row)
    x_meal.append(x1)
    y_meal.append(list(y1))
  # print(len(y_meal))
  peak_valdf_meal = pd.DataFrame(y_meal)
  return (peak_valdf_meal)

In [25]:
def windowed_mean_feature(meal_cgm_master_list):  
  def windowed_mean(meal_list,row):
    w_size = 5
    window_averages =[]
    i=0
    while i < len(meal_list[row]) - w_size +1: 
      nw_size = meal_list[row][i : i + w_size]
      w_average = sum(nw_size)/w_size
      window_averages.append(w_average)
      i+=w_size
    # print(len(window_averages))
    return (window_averages[1:5]) 

  windowed_average_list_meal = []
  for row in range (len(data)):
    average_list = windowed_mean(data,row)
    windowed_average_list_meal.append(average_list)
  windowed_average_list_df_meal = pd.DataFrame(windowed_average_list_meal)
  return (windowed_average_list_df_meal)

In [26]:
def wavelet_transform_feature(meal_cgm_master_list):  
  def discrete_wavelet_trans(meal_data,row):
    (cA,cD) = pywt.dwt(meal_data[row], 'db1')
    cA = cA[::-1][0:8]
    return (cA,cD)

  x_meal=[]
  for row in range(len(data)):
    x1,y1 = discrete_wavelet_trans(data,row)
    x_meal.append(list(x1))
    discrete_wavelet_df_meal = pd.DataFrame(x_meal)  
  return (discrete_wavelet_df_meal) 

In [27]:
def Coeff_of_variation_feature(meal_cgm_master_list):
  def coeff_of_var(meal_data,row):
    mean = np.mean(meal_data[row])
    std_dev = np.std(meal_data[row])
    result = mean/std_dev
    return (result)

  coeff_of_var_meal_list = []
  for row in range(len(data)):
      coeff_val = coeff_of_var(data,row)
      coeff_of_var_meal_list.append(coeff_val)
  coeff_of_var_df_meal = pd.DataFrame(coeff_of_var_meal_list) 
  return (coeff_of_var_df_meal) 

In [28]:
def diff_max_min_feature(meal_cgm_master_list): 
  def diff_max_min_cgm(meal_list,row):
    max_cgm_val = max(meal_list[row])
    min_cgm_val = min(meal_list[row])
    result = max_cgm_val - min_cgm_val
    return (result)

  difference_list_meal = []
  for row in range (len(data)):
    difference = diff_max_min_cgm(data,row)
    difference_list_meal.append(difference)
  difference_df_meal = pd.DataFrame(difference_list_meal) 
  return (difference_df_meal)

In [29]:
df1= diff_max_min_feature(data)
df2=Coeff_of_variation_feature(data)
df3=wavelet_transform_feature(data)
df4=windowed_mean_feature(data)
df5=peaks_fouriertrans_feature(data)
df6=z_score_feature(data)
df7=rank_feature(data)








In [30]:
prediction_dataset = pd.concat([df1,df2,df3,df4,df5,df6,df7],axis=1).fillna(0)


In [31]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy import stats

feat_dataset = StandardScaler().fit_transform(prediction_dataset)

pca = PCA(n_components= 15)
pca_component_extract = pca.fit_transform(feat_dataset)
pca_dataframe = pd.DataFrame(data = pca_component_extract, columns = ['PC 1', 'PC 2','PC 3','PC 4','PC 5','PC 6','PC 7','PC 8','PC 9','PC 10','PC 11','PC 12','PC 13','PC 14','PC 15'])

In [32]:
# using from : https://stackoverflow.com/questions/56107259/how-to-save-a-trained-model-by-scikit-learn
from joblib import dump, load
with open('SVM_Classifier.pickle', 'rb') as loaded_model:
    pickle_file = load(loaded_model)
    prediction_metric = pickle_file.predict(pca_dataframe)    
    loaded_model.close()
    print('Prediction :-')
    print (prediction_metric)

Prediction :-
[1 1 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1
 0 0 0 0 1 1]


In [33]:
prediction_df = DataFrame(prediction_metric,columns=['Output Bits'])
prediction_df.to_csv('Results_Makhijani_2.csv',index=False)
print(prediction_df.head)

<bound method NDFrame.head of     Output Bits
0             1
1             1
2             0
3             1
4             1
5             1
6             1
7             0
8             0
9             0
10            0
11            1
12            0
13            1
14            1
15            1
16            0
17            1
18            0
19            1
20            1
21            1
22            1
23            0
24            1
25            1
26            1
27            1
28            1
29            1
30            1
31            1
32            1
33            0
34            1
35            0
36            1
37            0
38            0
39            0
40            0
41            1
42            1>
